### Universidad de San Carlos de Guatemala
### Facultad de Ingeniería
### Escuela de Estudios de Postgrado
### Curso: Mineria de Datos.
### Introducción a la Minería de Datos
### Por: Denys Fernando Orozco Escobar

# **Redes Neuronales**

Las redes neuronales son un tipo de modelo computacional que se inspira en el funcionamiento del cerebro humano, diseñado para aprender a partir de datos y tomar decisiones basadas en esos aprendizajes. A continuación, se presentan los aspectos más relevantes sobre las redes neuronales, su estructura, funcionamiento y aplicaciones.

 En plataformas como Google Colab, los investigadores y desarrolladores pueden implementar y entrenar redes neuronales utilizando potentes bibliotecas como TensorFlow, Keras y PyTorch, aprovechando recursos computacionales avanzados como GPU y TPU para acelerar el proceso de aprendizaje.

In [2]:
#Ejecute las librerías
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [7]:
#Abre una ventana de exploración para carga de documentos .sav
from google.colab import files
# Subir archivos .sav
uploaded = files.upload()

Saving _D_2015.sav to _D_2015.sav
Saving _D_2014.sav to _D_2014.sav
Saving _D_2013.sav to _D_2013.sav
Saving D_2021.sav to D_2021.sav
Saving D_2020.sav to D_2020.sav
Saving D_2019.sav to D_2019.sav
Saving D_2018.sav to D_2018.sav
Saving D_2017.sav to D_2017.sav
Saving D_2016.sav to D_2016.sav


In [11]:
!pip install pyreadstat
import pyreadstat

# Cargar archivos .sav
datos_2013_D , meta1 = pyreadstat.read_sav("_D_2013.sav")
datos_2014_D , meta2 = pyreadstat.read_sav("_D_2014.sav")
datos_2015_D , meta3 = pyreadstat.read_sav("_D_2015.sav")

# Visualizar los primeros registros
print(datos_2013_D.head())

   Depreg Mupreg  Mesreg  Añoreg  Depocu Mupocu  Areag  Sexo  Diaocu  Mesocu  \
0    13.0   1327    10.0  2013.0    13.0   1327    2.0   1.0     4.0    10.0   
1     1.0   0101     5.0  2013.0     1.0   0101    1.0   2.0    25.0     5.0   
2     8.0   0801     5.0  2013.0     8.0   0801    2.0   2.0    25.0     5.0   
3     6.0   0606     3.0  2013.0     6.0   0606    2.0   1.0    19.0     3.0   
4     1.0   0113     4.0  2013.0     1.0   0113    2.0   2.0     7.0     4.0   

   ...  Mnadif  Nacdif  Predif  Dredif  Mredif Caudef  \
0  ...    1327   320.0   320.0    13.0    1327   R000   
1  ...    2001   320.0   320.0     1.0    0101   R000   
2  ...    0801   320.0   320.0     8.0    0801   R000   
3  ...    0602   320.0  9999.0    99.0    9999   R000   
4  ...    0101   320.0  9999.0    99.0    9999   R011   

                    caudef.descrip  Asist Ocur  Cerdef  
0     Taquicardia, no especificada    1.0  6.0     9.0  
1     Taquicardia, no especificada    1.0  2.0     1.0  
2    

In [17]:
# Concatenar filas
datos_d= pd.concat([datos_2013_D, datos_2014_D, datos_2015_D], ignore_index=True)
# Verificar resultado
print(datos_d.head())

   Depreg Mupreg  Mesreg  Añoreg  Depocu Mupocu  Areag  Sexo  Diaocu  Mesocu  \
0    13.0   1327    10.0  2013.0    13.0   1327    2.0   1.0     4.0    10.0   
1     1.0   0101     5.0  2013.0     1.0   0101    1.0   2.0    25.0     5.0   
2     8.0   0801     5.0  2013.0     8.0   0801    2.0   2.0    25.0     5.0   
3     6.0   0606     3.0  2013.0     6.0   0606    2.0   1.0    19.0     3.0   
4     1.0   0113     4.0  2013.0     1.0   0113    2.0   2.0     7.0     4.0   

   ...  Nacdif  Predif  Dredif  Mredif  Caudef  \
0  ...   320.0   320.0    13.0    1327    R000   
1  ...   320.0   320.0     1.0    0101    R000   
2  ...   320.0   320.0     8.0    0801    R000   
3  ...   320.0  9999.0    99.0    9999    R000   
4  ...   320.0  9999.0    99.0    9999    R011   

                    caudef.descrip  Asist  Ocur Cerdef  Añoocu  
0     Taquicardia, no especificada    1.0   6.0    9.0     NaN  
1     Taquicardia, no especificada    1.0   2.0    1.0     NaN  
2     Taquicardia, no e

In [18]:
#Aleatoriedad
datos_d = datos_d.sample(frac = 1, random_state = 42).reset_index(drop=True)

In [19]:
#Selección de la data
X = datos_d[['Cerdef','Sexo','Escodif']]
y = datos_d['Depreg']

In [20]:
#Datos de entrenamiento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train = X_train.replace(' ', np.nan).dropna().astype('float32')
# Inicializando LabelEncoder
label_encoder = LabelEncoder()
# Fit and transform y_train
#Fit on all unique values to avoid unseen labels during transform
all_unique_labels = pd.concat([y_train[X_train.index], y_test[X_test.index]]).unique()
label_encoder.fit(all_unique_labels)
y_train = label_encoder.transform(y_train[X_train.index]).astype('float32')

X_test = X_test.replace(' ', np.nan).dropna().astype('float32')
# Transform y_test using the same encoder
y_test = label_encoder.transform(y_test[X_test.index]).astype('float32')

In [21]:
#Densidad de las capas, relu: función de activación de la capa (cambia valores negativos a positivos), introduce no linealidades
#sigmoid salida que representa la probabilidad de pertenencia a una de las clases
model = Sequential()
model.add(Dense(8, input_dim = 3, activation = 'relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [23]:
#Entrenamiento
model.fit(X_train, y_train, epochs = 50, batch_size = 200, validation_data=(X_test, y_test))


Epoch 1/50
942/942 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.0742 - loss: -217.4166 - val_accuracy: 0.0114 - val_loss: -4172.7139
Epoch 2/50
942/942 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0104 - loss: -8928.5273 - val_accuracy: 0.0114 - val_loss: -30835.2695
Epoch 3/50
942/942 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0103 - loss: -43570.4258 - val_accuracy: 0.0114 - val_loss: -89721.5000
Epoch 4/50
942/942 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0103 - loss: -112107.0156 - val_accuracy: 0.0114 - val_loss: -185876.5000
Epoch 5/50
942/942 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0102 - loss: -219850.8750 - val_accuracy: 0.0114 - val_loss: -322993.0938
Epoch 6/50
942/942 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.0103 - loss: -367640.8125 - val_accuracy: 0.0114 - val_loss: -505030.1562
Epoch 7/50
942/942 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.0105 - loss: -564668.3750 - val_accuracy: 0.0114 - val_loss: -735691.1250
Epoch 8/50
942/942 ━━━━━━━━━━━

Probar el porcentaje de asertividad del modelo

In [24]:
loss, acc = model.evaluate(X_test, y_test)
print(acc*100)

1471/1471 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0113 - loss: -112828960.0000
1.13885048776865
